In [ ]:
!wget https://www.cs.cmu.edu/~enron/enron_mail_20150507.tar.gz

--2021-02-22 21:17:41--  https://www.cs.cmu.edu/~enron/enron_mail_20150507.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 443254787 (423M) [application/x-gzip]
Saving to: ‘enron_mail_20150507.tar.gz’

enron_mail_20150507 100%[===================>] 422.72M  1.78MB/s    in 4m 3s   

2021-02-22 21:21:44 (1.74 MB/s) - ‘enron_mail_20150507.tar.gz’ saved [443254787/443254787]



In [ ]:
import tarfile
fname = '/content/enron_mail_20150507.tar.gz'

tar = tarfile.open(fname, "r:gz")
tar.extractall(path="/content/")
tar.close()


In [ ]:
import os
from pathlib import Path
import email
from email.policy import default
from email.parser import Parser
from collections import namedtuple

import pandas as pd
from bs4 import BeautifulSoup
from dateutil.parser import parse
from chardet.universaldetector import UniversalDetector

import pandas as pd

current_dir = Path(os.curdir)
data_dir = current_dir.joinpath('/content')
enron_data_dir = data_dir.joinpath('maildir')
enron_data_dir

PosixPath('/content/maildir')

In [ ]:
def read_raw_email(email_path):
    detector = UniversalDetector()
    
    try:
        with open(email_path) as f:
            original_msg = f.read()
    except UnicodeDecodeError:
        detector.reset()
        with open(email_path, 'rb') as f:
            for line in f.readlines():
                detector.feed(line)
                if detector.done:
                    break
        detector.close()
        encoding = detector.result['encoding']
        with open(email_path, encoding=encoding) as f:
            original_msg = f.read()
            
    return original_msg 
  
def make_df(data_dir):
    records = []
    count = 0
    for root, dirs, files in os.walk(data_dir):
        for file_path in files:
            current_path = Path(root).joinpath(file_path)
            count = count + 1
            username = str(current_path).split('/')[3]
            original_msg = read_raw_email(current_path)
            msg_id = str(current_path)[16:] 
            records.append([msg_id,username,original_msg])
    columns = ['id','username','original_msg']
    data = pd.DataFrame(records, columns=columns)        
    return data

In [ ]:
data = make_df(enron_data_dir)


In [ ]:
import string

def parse_html_payload(payload):
    """
    This function uses Beautiful Soup to read HTML data
    and return the text.  If the payload is plain text, then
    Beautiful Soup will return the original content
    """
    soup = BeautifulSoup(payload, 'html.parser')
    return str(soup.get_text()).encode('utf-8').decode('utf-8')

def parse_email(original_msg):
    parsed_msg = parse_html_payload(original_msg)
    
    result = {}
    msg = Parser(policy=default).parsestr(parsed_msg)
    
    if msg.is_multipart(): # incase the payload is nested further in the email, should help with error handling
        for payload in msg.get_payload():
            body = payload.get_payload()
    else:
        body = msg.get_payload()
    trans_str = string.punctuation + string.digits  
    body = body.translate(str.maketrans('', '', trans_str))
    body = " ".join(body.split())
    body = body.lower()
    return body

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data['email_body'] = data['original_msg'].apply(parse_email)

In [ ]:
data.to_csv('enron_processed.csv.gz', compression='gzip', chunksize = 1000)